In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2

import os
from random import shuffle
from glob import glob

from matplotlib import pyplot as plt

print(tf.__version__)
print(tf.executing_eagerly())

In [ ]:
IMG_SIZE = (224, 224)  # размер входного изображения сети

train_files = glob('.data/train_sample/*.jpg')
test_files = glob('.data/test_sample/*.jpg')

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

def load_image(path):
    img = cv2.imread(path)[..., ::-1]
    img = cv2.resize(img, IMG_SIZE)
    return preprocess_input(img)


In [ ]:
def fit_generator(files, batch_size=32):
    while True:
        shuffle(files)
        for i in range(0, len(files), batch_size):
            batch = files[i:i + batch_size]
            x = np.array([load_image(p) for p in batch])
            y = np.array([1. if os.path.basename(p).startswith('dog') else 0.
                          for p in batch])
            yield x, y


In [ ]:
def predict_generator(files):
    for path in files:
        yield np.array([load_image(path)])


##Визуализируем примеры для обучения

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
fig = plt.figure(figsize=(16, 8))
for i, path in enumerate(train_files[:10], 1):
    subplot = fig.add_subplot(2, 5, i)
    subplot.set_title('%s' % path.split('/')[-1])
    img = cv2.imread(path)[...,::-1]
    img = cv2.resize(img, IMG_SIZE)
    plt.imshow(img)

## Загружаем предобученную модель

In [ ]:
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)
)

In [ ]:
base_model.summary()

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=x)

## Выводим архитектуру модели

In [ ]:
model.summary()

## Компилируем модель и запускаем обучение

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
val_size = 2000

shuffle(train_files)
val_data = next(fit_generator(train_files[:val_size], val_size))
train_gen = fit_generator(train_files[val_size:], batch_size=32)

model.fit(
    train_gen,
    steps_per_epoch=100,
    epochs=10,
    validation_data=val_data
)


## Предсказания на проверочной выборке

In [ ]:
test_pred = model.predict(
    predict_generator(test_files),
    steps=len(test_files)
)


In [ ]:
fig = plt.figure(figsize=(12, 6))
for i, (path, score) in enumerate(zip(test_files[:6], test_pred[:6]), 1):
    ax = fig.add_subplot(2, 3, i)
    img = cv2.imread(path)[..., ::-1]
    img = cv2.resize(img, IMG_SIZE)
    ax.imshow(img)
    ax.set_title(f'{score[0]:.2f}')
    ax.axis('off')


## Готовим данные для отправки

In [ ]:
import re

with open('submit.txt', 'w') as f:
    f.write('id,label\n')
    for path, score in zip(test_files, test_pred):
        img_id = re.search(r'(\d+)\.jpg$', path).group(1)
        f.write(f'{img_id},{score[0]}\n')
